In [ ]:
from osgeo import ogr
from osgeo import gdal
import numpy as np
import pandas as pd
import geopandas as gpd
import critical_loads as cl
import imp

# Critical Loads Workflow

This notebook implements a new workflow for the Critical Loads processing. There are three main components: vegetation, water and soils.

## 1. Vegetation

A new raster-based workflow was developed in [this notebook](http://nbviewer.jupyter.org/github/JamesSample/critical_loads/blob/master/notebooks/critical_loads_vegetation.ipynb). The code below loops over all the datasets to create vegetation exceedance grids (in mg-N/l) for the time periods of interest.

### 1.1. Upload new data to database

In [ ]:
# Add code to upload new data to db. Located here:
# K:\Avdeling\317 Klima- og miljømodellering\KAU\Focal Centre\Projects\2017 Tålegrenseprosjekt\Data\dep1216_til_NIVA.xlsx

# Note some heading in cells below are currently hard-coded and will need adapting

### 1.2. Read lookup table link vegetation classes to critical loads

In [ ]:
# Read lookup table
in_xlsx = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
           r'\sat_veg_land_use_classes.xlsx')
df = pd.read_excel(in_xlsx, sheetname='EUNIS_tilGIS', index_col=0)

df = df[['CL_100smgN/m2/yr']].round(0).astype(int)

df.head()

### 1.3. Reclassify

In [ ]:
# Input national veg map
in_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
          r'\GIS\Raster\sat_veg_30m_all.tif')

# Output geotiff for critical loads values
out_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
           r'\GIS\Raster\sat_veg_30m_cr_lds_div100.tif')

# Reclassify
cl.reclassify_raster(in_tif, out_tif, df, 'CL_100smgN/m2/yr', 255)

### 1.4. Get deposition data from database

In [ ]:
# Connect to db
resa2_basic_path = (r'C:\Data\James_Work\Staff\Heleen_d_W\ICP_Waters\Upload_Template'
                    r'\useful_resa2_code.py')
resa2_basic = imp.load_source('useful_resa2_code', resa2_basic_path)
engine, conn = resa2_basic.connect_to_resa2()

In [ ]:
# Get all dep values
sql = ("SELECT blr, dep_series_id as series, value as dep "
       "FROM resa2.dep_blr_values "
       "WHERE parameter_id IN (1, 2) "
       "AND dep_series_id IN (1, 2, 3, 4, 25)")

df = pd.read_sql(sql, engine)

# Sum N_oks and N_red
df = df.groupby(['blr', 'series']).sum()

# Reshape and tidy
df = df.unstack()
df.columns = df.columns.get_level_values(1)
df.columns.name = ''
df.columns = ['Ndep78_82', 'Ndep92_96', 'Ndep97_01', 'Ndep02_06', 'Ndep07_11']

print np.nanmin(df.values), np.nanmax(df.values)

df.head()

### 1.5. Loop over datasets

In [ ]:
%%time

# Path to raw BLR shapefile
in_shp = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\GIS'
          r'\Shapefiles\blrgrid_uten_grums_utm_z33n.shp')

# Snap raster
snap_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
            r'\GIS\Raster\sat_veg_30m_all.tif')

# Critical loads raster
cl_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
          r'\GIS\Raster\sat_veg_30m_cr_lds_div100.tif')

# Container for output
data_dict = {'series':[],
             'nor_area_km2':[],
             'ex_area_km2':[]}

# Loop over series
for ser in ['Ndep78_82', 'Ndep92_96', 'Ndep97_01', 'Ndep02_06', 'Ndep07_11']:    
    
    print 'Processing: %s' % ser
    print '    Building deposition shapefile...'
    
    # Get deposition
    dep_df = df[[ser]].dropna(how='any').round(0).astype(int).reset_index()

    # Rename cols to match shapefile
    dep_df.columns = ['BLR', ser]

    # Read shapefile
    blr_df = gpd.read_file(in_shp)

    # Join and tidy
    dep_df = blr_df.merge(dep_df, on='BLR')
    del dep_df['area_m2']
    
    # Write output shapefile
    dep_shp = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\GIS'
               r'\Shapefiles\dep_%s.shp' % ser)
    dep_df.to_file(dep_shp)
    
    # Convert shp to ras
    print '    Rasterising shapefile...'
    
    # Output BLR raster
    dep_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
               r'\GIS\Raster\dep_%s_30m.tif' % ser)

    cl.vec_to_ras(dep_shp, dep_tif, snap_tif, ser, -1, gdal.GDT_Int16)
    
    # Exceedance
    print '    Calculating exceedance...'
    
    # Read grids
    cl_grid, cl_ndv = cl.read_geotiff(cl_tif)
    dep_grid, dep_ndv = cl.read_geotiff(dep_tif)

    # Upcast to float32 for safe handling of negative values
    cl_grid = cl_grid.astype(np.float32)
    dep_grid = dep_grid.astype(np.float32)
   
    # Set ndv
    cl_grid[cl_grid==cl_ndv] = np.nan
    dep_grid[dep_grid==dep_ndv] = np.nan

    # Get total area of non-NaN from dep grid
    nor_area = np.count_nonzero(~np.isnan(dep_grid))*30.*30./1.E6

    # Apply scaling factor to CLs
    cl_grid = cl_grid*100.

    # Exceedance
    ex_grid = dep_grid - cl_grid
    del dep_grid, cl_grid  
    
    # Get total area exceeded
    ex_area = np.count_nonzero(ex_grid > 0)*30.*30./1.E6

    # Set <0 to 0
    ex_grid[ex_grid<0] = 0
    
    # Reset ndv
    ex_grid[np.isnan(ex_grid)] = -1

    # Downcast to int16 to save space
    ex_grid = ex_grid.round(0).astype(np.int16)
    
    # Append results
    data_dict['series'].append(ser)
    data_dict['nor_area_km2'].append(nor_area)
    data_dict['ex_area_km2'].append(ex_area)
    
    # Write output
    print '    Saving exceedance grid...'
    ex_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
              r'\GIS\Raster\exceed_%s_30m.tif' % ser)
    
    cl.write_geotiff(ex_grid, ex_tif, snap_tif, -1, gdal.GDT_Int16)
    
    print '    Done.'

# Build output df
ex_df = pd.DataFrame(data_dict)
ex_df['ex_pct'] = 100 * ex_df['ex_area_km2'] / ex_df['nor_area_km2']
ex_df.index = ex_df['series']
del ex_df['series']
ex_df = ex_df.round(0).astype(int)

# Save
out_csv = r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\nor_prop_exceed.csv'
ex_df.to_csv(out_csv)

print 'Finished.'

### 1.6. Print national exceedance summary for vegetation

In [ ]:
# Exceedance summary
ex_df